In [36]:
import pandas as pd
import numpy as np
import multiprocessing
from scipy import stats
import timeit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import validation_curve
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import scale
%matplotlib inline
import matplotlib.pyplot as plt
#import aux_functions
#from aux_functions import plot_with_err

# Import data

##Train

In [37]:
X = pd.read_csv('/content/cluster_train.csv')
y = X['good_bad_flag']
del X['good_bad_flag']
X.head()

,loanamount,interest,avg_loanamount,avg_paym_span_hrs,avg_active_span_days,avg_interest,max_loannumber,age,interest_interval_3000,interest_interval_mas de 3000,interest_interval_menos de 3000,bank_name_clients_Multinational Banks,termdays_Other,age_interval_26-35,age_interval_36-40,age_interval_41-61,state_Other,loan_interval_mas de 10000,loannumber_0-4,cluster
0,10000.0,3000.0,10000.000000,0.000000,33.000000,3000.000000,1.0,36.0,1,0,0,1,0,0,1,0,0,0,1,0
1,10000.0,3000.0,10000.000000,13.000000,3.000000,3000.000000,1.0,41.0,1,0,0,0,0,0,0,1,0,0,1,0
2,20000.0,3000.0,15000.000000,15.166667,23.333333,3133.333333,6.0,34.0,1,0,0,1,0,1,0,0,1,1,0,0
3,20000.0,4500.0,10000.000000,13.666667,25.333333,3000.000000,3.0,47.0,0,1,0,0,0,0,0,1,1,1,1,0
4,20000.0,4500.0,12727.272727,11.636364,14.000000,1559.090909,11.0,27.0,0,1,0,1,0,1,0,0,1,1,0,0


Tipificamos

In [38]:
X_scale = pd.DataFrame(scale(X))
X_scale.columns = X.columns
X = X_scale
X.columns = X_scale.columns
X.head()

,loanamount,interest,avg_loanamount,avg_paym_span_hrs,avg_active_span_days,avg_interest,max_loannumber,age,interest_interval_3000,interest_interval_mas de 3000,interest_interval_menos de 3000,bank_name_clients_Multinational Banks,termdays_Other,age_interval_26-35,age_interval_36-40,age_interval_41-61,state_Other,loan_interval_mas de 10000,loannumber_0-4,cluster
0,-0.733215,-0.285638,-0.608786,-3.179329,1.276814,0.330176,-0.876227,-0.060081,1.260701,-0.783287,-0.552029,0.806211,-0.613458,-0.748083,1.456264,-0.687158,-1.065516,-0.890780,0.884886,-0.560283
1,-0.733215,-0.285638,-0.608786,0.004068,-2.268915,0.330176,-0.876227,-0.059547,1.260701,-0.783287,-0.552029,-1.240371,-0.613458,-0.748083,-0.686689,1.455270,-1.065516,-0.890780,0.884886,-0.560283
2,0.194972,-0.285638,0.439597,0.534634,0.134301,0.491189,0.487956,-0.060294,1.260701,-0.783287,-0.552029,0.806211,-0.613458,1.336749,-0.686689,-0.687158,0.938513,1.122611,-1.130089,-0.560283
3,0.194972,0.600455,-0.608786,0.167319,0.370683,0.330176,-0.330554,-0.058906,-0.793209,1.276672,-0.552029,-1.240371,-0.613458,-0.748083,-0.686689,1.455270,0.938513,1.122611,0.884886,-0.560283
4,0.194972,0.600455,-0.036941,-0.329855,-0.968814,-1.409862,1.852139,-0.061042,-0.793209,1.276672,-0.552029,0.806211,-0.613458,1.336749,-0.686689,-0.687158,0.938513,1.122611,-1.130089,-0.560283


In [39]:
#dimensión del conjunto de train
n_train = int(X.shape[0])

#selección del conjunto de train
X_train = X.iloc[:n_train]
y_train = y.iloc[:n_train]

## Validation y Test

In [41]:
data = pd.read_csv('/content/cluster_test.csv')
X_t = data.drop(['customerid'],axis=1)
y_t = X_t['good_bad_flag']
del X_t['good_bad_flag']
X_t.head()

,loanamount,interest,avg_loanamount,avg_paym_span_hrs,avg_active_span_days,avg_interest,max_loannumber,age,interest_interval_3000,interest_interval_mas de 3000,interest_interval_menos de 3000,bank_name_clients_Multinational Banks,termdays_Other,age_interval_26-35,age_interval_36-40,age_interval_41-61,state_Other,loan_interval_mas de 10000,loannumber_0-4,cluster
0,20000.0,4500.0,14000.0,12.8,19.0,2700.0,5.0,39.0,0,1,0,1,0,0,1,0,1,1,0,0
1,10000.0,3000.0,10000.0,13.0,29.0,3000.0,2.0,42.0,1,0,0,1,0,0,0,1,1,0,1,0
2,10000.0,3000.0,10000.0,7.0,57.0,3000.0,1.0,45.0,1,0,0,0,0,0,0,1,1,0,1,0
3,10000.0,3000.0,10000.0,18.0,14.0,1500.0,1.0,40.0,1,0,0,1,0,0,1,0,0,0,1,0
4,20000.0,4500.0,14000.0,9.0,19.6,2700.0,5.0,37.0,0,1,0,1,0,0,1,0,0,1,0,0


Tipificamos

In [42]:
X_t_scale = pd.DataFrame(scale(X_t))
X_t_scale.columns = X_t.columns
X_t = X_t_scale
X_t.columns = X_t_scale.columns
X_t.head()

,loanamount,interest,avg_loanamount,avg_paym_span_hrs,avg_active_span_days,avg_interest,max_loannumber,age,interest_interval_3000,interest_interval_mas de 3000,interest_interval_menos de 3000,bank_name_clients_Multinational Banks,termdays_Other,age_interval_26-35,age_interval_36-40,age_interval_41-61,state_Other,loan_interval_mas de 10000,loannumber_0-4,cluster
0,0.232470,0.678767,0.257389,-0.084439,-0.289372,0.072892,0.293053,-0.048716,-0.755045,1.310895,-0.606694,0.758937,-0.662085,-0.770667,1.468977,-0.671399,0.928724,1.113553,-1.189030,-0.535136
1,-0.736634,-0.233861,-0.596311,-0.036304,0.865412,0.425222,-0.566660,-0.048324,1.324424,-0.762837,-0.606694,0.758937,-0.662085,-0.770667,-0.680746,1.489426,0.928724,-0.898027,0.841021,-0.535136
2,-0.736634,-0.233861,-0.596311,-1.480364,4.098808,0.425222,-0.853231,-0.047932,1.324424,-0.762837,-0.606694,-1.317633,-0.662085,-0.770667,-0.680746,1.489426,0.928724,-0.898027,0.841021,-0.535136
3,-0.736634,-0.233861,-0.596311,1.167079,-0.866764,-1.336431,-0.853231,-0.048585,1.324424,-0.762837,-0.606694,0.758937,-0.662085,-0.770667,1.468977,-0.671399,-1.076747,-0.898027,0.841021,-0.535136
4,0.232470,0.678767,0.257389,-0.999010,-0.220085,0.072892,0.293053,-0.048978,-0.755045,1.310895,-0.606694,0.758937,-0.662085,-0.770667,1.468977,-0.671399,-1.076747,1.113553,-1.189030,-0.535136


Dividimos el dataset de test en validation y test, es decir, 50-50. Representarán el 10% del dataset total cada uno, ya que el train supone un 80% del total de los datos.

In [43]:
perc_values = [0.5, 0.5]

# dimensiones de los conjuntos de validation y test
n_val = int(X_t.shape[0] * perc_values[0])
n_test = int(X_t.shape[0] * perc_values[1])

# selección del conjunto de validación
X_val = X_t.iloc[:n_val]
y_val = y_t.iloc[:n_val]

# selección del conjunto de test
X_test = X_t.iloc[n_test:]
y_test = y_t.iloc[n_test:]

In [44]:
#Visualizamos el tamaño de los tres tipos de datasets

print('Train data size = ' + str(X_train.shape))
print('Train target size = ' + str(y_train.shape))
print('Validation data size = ' + str(X_val.shape))
print('Validation target size = ' + str(y_val.shape))
print('Test data size = ' + str(X_test.shape))
print('Test target size = ' + str(y_test.shape))

Train data size = (3361, 20)
Train target size = (3361,)
Validation data size = (420, 20)
Validation target size = (420,)
Test data size = (420, 20)
Test target size = (420,)


#Search Grid

In [45]:
#Hemos decidido probar los siguientes modelos:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [46]:
#Importamos la métrica que evaluará nuestros modelos
from sklearn.metrics import roc_auc_score as metric

## Parámetros generales

In [47]:
random_state = 42    # inicializa el random number generator interno
nthread = multiprocessing.cpu_count() - 1
scale_pos_weight = sum(y_train == 0) / sum(y_train == 1)

In [48]:
#puntos recomendados por Cherkassky para la SVM

n = X_train.shape[0]
d = X_train.shape[1]
m = np.mean(y_train)
s = np.std(y_train)
C_cherk = np.max([np.abs(m + 3*s),np.abs(m - 3*s)])
gamma_cherk = np.power(0.2, 1/d)


## Definición del grid

In [49]:
# Regresion Logística
regularization_values = ['l1', 'l2', 'none']   #reduce overfitting
penalty_values = [1, 10, 100]   #penaliza por tener demasiadas variables

# SVM
C_values = [C_cherk, 5e-03, 4.5e-03, 4e-03]    #parámetro de regularización 
gamma_kernel_values = [gamma_cherk, 3.26e-09, 3.255e-09, 3.25e-09]    #influencia del train

# Random Forest
ntrees_values = [10, 100, 1000]
max_depth_values = [None, 6, 20];
min_samples_split_values = [2, 5, 20];
min_samples_leaf_values = [1, 5, 20];
max_features_values = [None, 1, 2];

# Xgboost
nrounds_values = [10, 100]
eta_values = [0.3, 0.99]
gamma_values = [0, 1]
max_depth_values = [6, 20]
min_child_weight_values = [1, 20]
subsample_values = [0.1, 1]
colsample_bytree_values = [0.1, 1]
num_parallel_tree_values = [1, 20]
lambda_values = [0, 1]
alpha_values = [0, 1]

In [50]:
params_values = [{'model': 'logistic regression',
                  'regularization': regularization_values,
                 'penalty': penalty_values},

                 {'model': 'svm',
                  'C': C_values,
                 'gamma_kernel': gamma_kernel_values},

                 {'model': 'random forest',
                  'n_trees': ntrees_values,
                 'max_depth': max_depth_values,
                 'min_samples_split': min_samples_split_values,
                 'min_samples_leaf': min_samples_leaf_values,
                 'max_features': max_features_values},

                 {'model': 'xgboost',
                  'nrounds': nrounds_values,
                  'eta': eta_values,
                 'gamma': gamma_values,
                 'max_depth': max_depth_values,
                 'min_child_weight': min_child_weight_values,
                 'subsample': subsample_values,
                 'colsample_bytree': colsample_bytree_values,
                 'num_parallel_tree': num_parallel_tree_values,
                 'lambda': lambda_values,
                 'alpha': alpha_values}]

In [51]:
total_iteraciones = 0
for params in params_values:
    if params['model'] == 'logistic regression':
        n = len(params['regularization'])*len(params['penalty'])
    elif params['model'] == 'svm':
        n = len(params['C'])*len(params['gamma_kernel'])
    elif params['model'] == 'random forest':
        n = len(params['n_trees'])*len(params['max_depth'])*len(params['min_samples_split'])*len(params['min_samples_leaf'])*len(params['max_features'])
    elif params['model'] == 'xgboost':
        n = len(params['nrounds'])*len(params['eta'])*len(params['gamma'])*len(params['max_depth'])*len(params['min_child_weight'])*len(params['subsample'])*len(params['colsample_bytree'])*len(params['num_parallel_tree'])*len(params['lambda'])*len(params['alpha'])
    total_iteraciones = total_iteraciones + n;
    print(str(n)+ ' iteraciones de ' + str(params['model']))
print(str(total_iteraciones)+ ' iteraciones en total')        

9 iteraciones de logistic regression
16 iteraciones de svm
162 iteraciones de random forest
1024 iteraciones de xgboost
1211 iteraciones en total


## Aplicamos modelos

In [52]:
grid_results = pd.DataFrame();
num_iter = 0
for params in params_values:
  
    #Regresion Logística
    if params['model'] == 'logistic regression':
        for regularization in params['regularization']:
            for penalty in params['penalty']:  
                start = timeit.default_timer()
                num_iter += 1
                print('Inicio de iteracion ' + str(num_iter) + '. Regularizacion = ' + str(regularization) + ', Lambda = '  + str(penalty) + '\n')
                
                #Entrenar modelo
                if regularization == 'l1':
                    model = LogisticRegression(penalty = regularization, solver = 'liblinear', C = penalty, random_state = random_state)
                else:
                    model = LogisticRegression(penalty = regularization,solver = 'lbfgs', C = penalty, random_state = random_state)
               
                model.fit(X_train, np.array(y_train))

                #Generar predicciones
                pred_train_p = model.predict_proba(X_train)
                pred_val_p = model.predict_proba(X_val)

                #Calcular métricas de evaluación
                auc_train = metric(y_train, pred_train_p[:, 1])
                auc_val = metric(y_val, pred_val_p[:, 1])
                time = timeit.default_timer() - start

                print('Fin de iteracion ' + str(num_iter) + '. Regularizacion = ' + str(regularization) + ', Lambda = '  + str(penalty) +
                      '. AUC train = '  + str(auc_train) + ' -  AUC val = '  + str(auc_val)  + ' -  time = '  + str(time)  + '\n')
                
                grid_results = grid_results.append(pd.DataFrame(data={'model':'Logistic Regression',
                                                                      'params': [{'regularization':[regularization], 'penalty':[penalty]}],
                                                                      'auc_train':[auc_train],
                                                                      'auc_val':[auc_val],
                                                                      'time':[time]},
                                                               columns=['model','params', 'auc_train', 'auc_val', 'time']), ignore_index=True)
                
    #SVM
    if params['model'] == 'svm':
        for C in params['C']:
            for gamma_kernel in params['gamma_kernel']:  
                start = timeit.default_timer()
                num_iter += 1
                print('Inicio de iteracion ' + str(num_iter) + '. C = ' + str(C) + ', gamma = '  + str(gamma_kernel) + '\n')
                
                #Entrenar modelo               
                model = SVC(C = C, gamma = gamma_kernel, probability = True, random_state = random_state)
                model.fit(X_train, np.array(y_train))

                #Generar predicciones
                pred_train_p = model.predict_proba(X_train)
                pred_val_p = model.predict_proba(X_val)

                #Calcular métricas de evaluación
                auc_train = metric(y_train, pred_train_p[:, 1])
                auc_val = metric(y_val, pred_val_p[:, 1])
                time = timeit.default_timer() - start

                print('Fin de iteracion ' + str(num_iter) + '. C = ' + str(C) + ', gamma = '  + str(gamma_kernel) + 
                      '. AUC train = '  + str(auc_train) + ' -  AUC val = '  + str(auc_val)  + ' -  time = '  + str(time)  + '\n')
                
                grid_results = grid_results.append(pd.DataFrame(data={'model':'SVM',
                                                                     'params': [{'C':[C], 'gamma_kernel':[gamma_kernel]}],
                                                                      'auc_train':[auc_train], 'auc_val':[auc_val], 'time':[time]},
                                                               columns=['model','params', 'auc_train', 'auc_val', 'time']), ignore_index=True)
                

    #Random Forest
    if params['model'] == 'random forest':
        for n_trees in params['n_trees']:
            for max_depth in params['max_depth']:
                for min_samples_split in params['min_samples_split']:  
                    for min_samples_leaf in params['min_samples_leaf']:  
                        for max_features in params['max_features']:  
                            start = timeit.default_timer()
                            num_iter += 1 
                            print('Inicio de iteracion ' + str(num_iter) + '. n_trees = ' + str(n_trees) + ', max_depth = ' + str(max_depth) + 
                                  ', min_samples_split = '  + str(min_samples_split) + ', min_samples_leaf = '  + str(min_samples_leaf) +
                                  ', max_features = '  + str(max_features) + '\n')

                            #Entrenar modelo               
                            model = RandomForestClassifier(n_estimators = n_trees, max_depth = max_depth,
                                                          min_samples_split = min_samples_split,
                                                          min_samples_leaf = min_samples_leaf,
                                                          max_features = max_features, random_state = random_state)

                            model.fit(X_train, np.array(y_train))

                            #Generar predicciones
                            pred_train_p = model.predict_proba(X_train)
                            pred_val_p = model.predict_proba(X_val)

                            #Calcular métricas de evaluación
                            auc_train = metric(y_train, pred_train_p[:, 1])
                            auc_val = metric(y_val, pred_val_p[:, 1])  
                            time = timeit.default_timer() - start

                            print('Fin de iteracion ' + str(num_iter) + '. n_trees = ' + str(n_trees) + ', max_depth = ' + str(max_depth) + 
                                  ', min_samples_split = '  + str(min_samples_split) + ', min_samples_leaf = '  + str(min_samples_leaf) +
                                  ', max_features = '  + str(max_features) + '. AUC train = '  + str(auc_train) +  ' -  AUC val = '  + str(auc_val)  + 
                                  ' -  time = '  + str(time)  + '\n')
                            
                            grid_results = grid_results.append(pd.DataFrame(data={'model':'random forest',
                                                                                  'params': [{'n_trees':[n_trees],'max_depth':[max_depth],
                                                                                              'min_samples_split':[min_samples_split],
                                                                                              'min_samples_leaf':[min_samples_leaf],
                                                                                              'max_features':[max_features]}],
                                                                                  'auc_train':[auc_train], 
                                                                                  'auc_val':[auc_val],
                                                                                  'time':[time]},
                                                                           columns=['model','params', 'auc_train', 'auc_val', 'time']), ignore_index=True)  
    #XGBOOST
    if params['model'] == 'xgboost':
         for nrounds in params['nrounds']:
            for eta in params['eta']:
                for gamma in params['gamma']:
                    for max_depth in params['max_depth']:
                        for min_child_weight in params['min_child_weight']:
                            for subsample in params['subsample']:
                                for colsample_bytree in params['colsample_bytree']:
                                    for num_parallel_tree in params['num_parallel_tree']:
                                        for lamda in params['lambda']:
                                            for alpha in params['alpha']:
                                                start = timeit.default_timer()
                                                num_iter += 1; 
                                                print('Inicio de iteracion ' + str(num_iter) + '. Parametro nrounds = ' + str(nrounds) + 
                                                      ', parametro eta = '  + str(eta) + ', parametro gamma = '  + str(gamma) +
                                                      ', parametro max_depth = '  + str(max_depth) + ', parametro min_child_weight = '  + str(min_child_weight) +
                                                      ', parametro subsample = '  + str(subsample) + ', parametro colsample_bytree = '  + str(colsample_bytree) +
                                                      ', parametro num_parallel_tree = '  + str(num_parallel_tree) + ', parametro lambda = '  + str(lamda) +
                                                      ', parametro alpha = '  + str(alpha) + '\n')
                                               
                                                #Entrenar modelo
                                                model = XGBClassifier(nthread = nthread, scale_pos_weight = scale_pos_weight,
                                                                      random_state = random_state, n_estimators = nrounds,
                                                                      learning_rate = eta, gamma = gamma,
                                                                      max_depth = max_depth, min_child_weight = min_child_weight ,
                                                                      subsample = subsample, colsample_bytree = colsample_bytree,
                                                                      num_parallel_tree = num_parallel_tree,
                                                                      reg_lambda = lamda, reg_alpha = alpha)
                                                model.fit(X_train, np.array(y_train))

                                                #Generar predicciones
                                                pred_train_p = model.predict_proba(X_train)
                                                pred_val_p = model.predict_proba(X_val)

                                                #Calcular métricas de evaluación
                                                auc_train = metric(y_train, pred_train_p[:, 1])
                                                auc_val = metric(y_val, pred_val_p[:, 1])    
                                                time = timeit.default_timer() - start

                                                print('Fin de iteracion ' + str(num_iter) + '. Parametro nrounds = ' + str(nrounds) + 
                                                      ', parametro eta = ' + str(eta) + ', parametro gamma = '  + str(gamma) +
                                                      ', parametro max_depth = '  + str(max_depth) + ', parametro min_child_weight = '  + str(min_child_weight) +
                                                      ', parametro subsample = '  + str(subsample) + ', parametro colsample_bytree = '  + str(colsample_bytree) +
                                                      ', parametro num_parallel_tree = '  + str(num_parallel_tree) + ', parametro lambda = '  + str(lamda) +
                                                      ', parametro alpha = '  + str(alpha) + '. AUC train = '  + str(auc_train) + 
                                                      ' -  AUC val = '  + str(auc_val)  + ' -  time = '  + str(time)  + '\n')
                                                grid_results = grid_results.append(pd.DataFrame(data={'model':'xgboost',
                                                                                                      'params': [{'nrounds':[nrounds], 'eta':[eta],
                                                                                                                  'gamma':[gamma], 'max_depth':[max_depth],
                                                                                                                  'min_child_weight':[min_child_weight],
                                                                                                                  'subsample':[subsample], 
                                                                                                                  'colsample_bytree':[colsample_bytree],
                                                                                                                  'num_parallel_tree':[num_parallel_tree],
                                                                                                                  'lamda':[lamda], 'alpha':[alpha]}],
                                                                                                      'auc_train':[auc_train], 'auc_val':[auc_val], 'time':[time]},
                                                                                                columns=['model', 'params', 'auc_train', 'auc_val', 'time']), ignore_index=True)

Inicio de iteracion 1. Regularizacion = l1, Lambda = 1

Fin de iteracion 1. Regularizacion = l1, Lambda = 1. AUC train = 0.7014891162139053 -  AUC val = 0.7397642286728355 -  time = 0.047803057999772136

Inicio de iteracion 2. Regularizacion = l1, Lambda = 10

Fin de iteracion 2. Regularizacion = l1, Lambda = 10. AUC train = 0.7015730753778162 -  AUC val = 0.7402078843959945 -  time = 0.13172896600008244

Inicio de iteracion 3. Regularizacion = l1, Lambda = 100



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


Fin de iteracion 3. Regularizacion = l1, Lambda = 100. AUC train = 0.7019810241987823 -  AUC val = 0.7400177462289264 -  time = 2.4135847279999325

Inicio de iteracion 4. Regularizacion = l2, Lambda = 1

Fin de iteracion 4. Regularizacion = l2, Lambda = 1. AUC train = 0.7015215666883003 -  AUC val = 0.7395107111167449 -  time = 0.047737740999764355

Inicio de iteracion 5. Regularizacion = l2, Lambda = 10

Fin de iteracion 5. Regularizacion = l2, Lambda = 10. AUC train = 0.7015622585530179 -  AUC val = 0.7400494359234376 -  time = 0.06856760699974984

Inicio de iteracion 6. Regularizacion = l2, Lambda = 100

Fin de iteracion 6. Regularizacion = l2, Lambda = 100. AUC train = 0.7015689546826549 -  AUC val = 0.7401761947014831 -  time = 0.08968855699959022

Inicio de iteracion 7. Regularizacion = none, Lambda = 1

Fin de iteracion 7. Regularizacion = none, Lambda = 1. AUC train = 0.7041727189376847 -  AUC val = 0.735581188997338 -  time = 0.0992184140000063

Inicio de iteracion 8. Regulari

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1484: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio parameters"
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as 

Fin de iteracion 8. Regularizacion = none, Lambda = 10. AUC train = 0.7041727189376847 -  AUC val = 0.735581188997338 -  time = 0.10256807899986597

Inicio de iteracion 9. Regularizacion = none, Lambda = 100

Fin de iteracion 9. Regularizacion = none, Lambda = 100. AUC train = 0.7041727189376847 -  AUC val = 0.735581188997338 -  time = 0.12041008500000316

Inicio de iteracion 10. C = 2.0232209027962953, gamma = 0.9226808345905884



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Fin de iteracion 10. C = 2.0232209027962953, gamma = 0.9226808345905884. AUC train = 0.9467044740447713 -  AUC val = 0.6435226264418811 -  time = 4.946357841999998

Inicio de iteracion 11. C = 2.0232209027962953, gamma = 3.26e-09

Fin de iteracion 11. C = 2.0232209027962953, gamma = 3.26e-09. AUC train = 0.6490213348991973 -  AUC val = 0.6894409937888197 -  time = 2.0605526459999055

Inicio de iteracion 12. C = 2.0232209027962953, gamma = 3.255e-09

Fin de iteracion 12. C = 2.0232209027962953, gamma = 3.255e-09. AUC train = 0.6231469748946648 -  AUC val = 0.6609519584231208 -  time = 2.072000110999852

Inicio de iteracion 13. C = 2.0232209027962953, gamma = 3.25e-09

Fin de iteracion 13. C = 2.0232209027962953, gamma = 3.25e-09. AUC train = 0.6487086771538358 -  AUC val = 0.6820572949676765 -  time = 2.0402980569997453

Inicio de iteracion 14. C = 0.005, gamma = 0.9226808345905884

Fin de iteracion 14. C = 0.005, gamma = 0.9226808345905884. AUC train = 0.9180939724531527 -  AUC val = 0

## Resultados

In [53]:
#por tiempo
grid_results.groupby(['model'], sort=False)['time'].mean().sort_values()

model
Logistic Regression    0.346812
xgboost                1.344198
random forest          2.061215
SVM                    2.491132
Name: time, dtype: float64

In [54]:
#por el AUC
grid_results.groupby(['model'], sort=False)['auc_val'].max().sort_values()

model
SVM                    0.707187
Logistic Regression    0.740208
random forest          0.758049
xgboost                0.769901
Name: auc_val, dtype: float64

In [55]:
#Elegimos la mejor combinación de modelo + hiperparámetros
best_params = grid_results.iloc[grid_results['auc_val'].idxmax()]

# Entrenamiento del modelo final

In [56]:
print('Train data size = ' + str(X_train.shape))
print('Train target size = ' + str(y_train.shape))
print('Validation data size = ' + str(X_val.shape))
print('Validation target size = ' + str(y_val.shape))

#Combinamos train y validación
X_train = pd.concat((X_train,X_val), axis = 0)
y_train = np.concatenate((y_train, y_val), axis = 0)

del X_val, y_val

print('Train data size = ' + str(X_train.shape))
print('Train target size = ' + str(y_train.shape))

Train data size = (3361, 20)
Train target size = (3361,)
Validation data size = (420, 20)
Validation target size = (420,)
Train data size = (3781, 20)
Train target size = (3781,)


In [57]:
#Regresión Logística
if best_params['model'] == 'logistic regression':       

    #Entrenar modelo
    if best_params['params']['regularization'] == 'l1':
        model = LogisticRegression(penalty = best_params['params']['regularization'][0], 
                                   solver = 'liblinear', C = best_params['params']['penalty'][0], random_state = random_state)
    else:
        model = LogisticRegression(penalty = best_params['params']['regularization'][0],
                                   solver = 'lbfgs', C = best_params['params']['penalty'][0], random_state = random_state)


#SVM
elif best_params['model'] == 'SVM':

    model = SVC(C = best_params['params']['C'][0], gamma = best_params['params']['gamma_kernel'][0], 
                probability = True, random_state = random_state)             


#Random Forest
elif best_params['model'] == 'random forest':
    model = RandomForestClassifier(n_estimators = int(best_params['params']['n_trees'][0]), max_depth = int(best_params['params']['max_depth'][0]),
                                                      min_samples_split = int(best_params['params']['min_samples_split'][0]),
                                                      min_samples_leaf = int(best_params['params']['min_samples_leaf'][0]),
                                                      max_features = int(best_params['params']['max_features'][0]), random_state = random_state)


# XGBOOST
elif best_params['model'] == 'xgboost':
    model = XGBClassifier(nthread = nthread, scale_pos_weight = scale_pos_weight, random_state = random_state,
                          n_estimators = int(best_params['params']['nrounds'][0]), learning_rate = best_params['params']['eta'][0], 
                          gamma = best_params['params']['gamma'][0], max_depth = int(best_params['params']['max_depth'][0]),
                          min_child_weight = best_params['params']['min_child_weight'][0], subsample = best_params['params']['subsample'][0],
                          colsample_bytree = best_params['params']['colsample_bytree'][0],
                          num_parallel_tree  = int(best_params['params']['num_parallel_tree'][0]),
                          reg_lambda = best_params['params']['lamda'][0], reg_alpha = best_params['params']['alpha'][0])

#Entrenar modelo
model.fit(X_train, np.array(y_train))

#Generar predicciones
pred_train_p = model.predict_proba(X_train)
pred_test_p = model.predict_proba(X_test)

#Calcular métricas de evaluación
auc_train = metric(y_train, pred_train_p[:, 1])
auc_test = metric(y_test, pred_test_p[:, 1]) 

results = pd.DataFrame()
results = results.append(pd.DataFrame(data={'model':best_params['model'],'auc_train':[auc_train],
                                            'auc_test':[auc_test]}, columns=['model',  'auc_train', 'auc_test']), ignore_index=True)

In [58]:
results

,model,auc_train,auc_test
0,xgboost,0.771737,0.659341
